In [ ]:
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install pandas --quiet

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

filename = "../raw_data/Search results 20220531.csv"

In [2]:
def read_metadata(filename):
    data = pd.read_csv(filename)
    data['content'] = None
    return data

def get_url_and_content(cellar_ref):
    psid = cellar_ref
    psname = "cellar" # other options: cellar, celex, oj, com, genpub, ep, jurisprudence, dd, mtf, consolidation, eurostat, eesc, cor, nim, pegase, agent, uriserv, join, swd, comnat,mdr, legissum, ecli, procedure, procedure-event, eli, immc, planjo
    lancode = "0006" # language code
    doctype = "03"
    docnum = "DOC_1"
    # for further information, see Documentation Page 37: https://op.europa.eu/en/publication-detail/-/publication/50ecce27-857e-11e8-ac6a-01aa75ed71a1/language-en/format-PDF/source-73059305
    URL = f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"

    response = requests.get(URL, headers={"Accept-Language":"en-US"})
    soup = BeautifulSoup(response.content, "html.parser")
    if str(soup)[1:4] == "PDF":
        '''
        in some (few) cases, the doctype is not 03 but 02. change it for these cases
        '''
        doctype = "02"
        URL = f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"
        response = requests.get(URL, headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
    else:
        pass
    return ' '.join([item.text for item in soup.find_all("p", class_="oj-normal")])

def append_content(filename):
    data = read_metadata(filename)
    cellar_references = data['Cellar reference']    
    for index, ref in enumerate(cellar_references):
        data['content'][index] = get_url_and_content(ref)
    return data

In [ ]:
data_with_content = append_content(filename)
data_with_content.to_csv("../raw_data/test_data_scraped.csv")

In [22]:
url_test= "https://eur-lex.europa.eu/search.html?SUBDOM_INIT=LEGISLATION&DB_TYPE_OF_ACT=regulation&DTS_SUBDOM=LEGISLATION&typeOfActStatus=REGULATION&DTS_DOM=EU_LAW&lang=en&type=advanced&qid=1653991199373"

In [23]:
response = requests.get(url_test)
soup = BeautifulSoup(response.content, "html.parser")